In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
daily = pd.read_csv('daily-aggregate.csv')
daily['date'] = pd.to_datetime(daily['name'], format="%Y-%m-%d")
daily = daily.set_index('date').asfreq('D')
daily = daily.drop(columns=['from', 'to', 'name'])

daily['Year'] = daily.index.year
daily['Month'] = daily.index.month
daily['Dag'] = daily.index.day_name()

# Reorder columns for convenience
daily = daily.reindex(columns=['Year', 'Month', 'Dag'] + daily.columns[:-3].tolist())

# Define (dutch) names
daily.columns = [
    'Jaar',
    'Maand',
    'Weekdag',
    'Gas',
    'Teruglevering',
    'Net_gebruik',
    'Bruto_productie',
    'Direct_gebruik_productie',
    'Direct_gebruik_productie_pct',
    'Eigen_gebruik_productie_pct',
    'Totaal_gebruik',
    'Terugleversaldo'
]

weekly_mean = daily.drop(columns=['Jaar', 'Maand']).resample('W').mean()
monthly_mean = daily.drop(columns=['Jaar', 'Maand']).resample('M').mean()

# Todo: derive from the dataset
years = ['2018', '2019', '2020']

In [ ]:
sns.set(rc={'figure.figsize':(18, 6)})

def compare_years(source, column, years, unit = 'kWh', lbl = lambda y, d: y, x_axis = lambda d: d.index.dayofyear):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, column]
        if unit == 'kWh':
            x = x.map(lambda x: x / 1000, na_action=None)
        ax.plot(x_axis(x), x, marker='x', linestyle='-', linewidth=0.5, label=lbl(year, x))
    ax.set_ylabel(column + ' ' + unit)
    ax.legend()

def compare_years_cumsum(source, column, years, unit = 'kWh'):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, column]
        x = x.cumsum(axis=0)
        if unit == 'kWh':
            x = x.map(lambda x: x / 1000, na_action=None)
        lbl = year + ' end: ' + str(round(x.iloc[-1], 2)) + ' ' + unit
        ax.plot(x.index.dayofyear, x, marker='x', linestyle='-', linewidth=0.5, label=lbl)
    ax.set_ylabel(column + ' ' + unit)
    ax.legend()

In [ ]:
# Running totals
compare_years_cumsum(daily, 'Gas', years, unit = 'm3')
compare_years_cumsum(daily, 'Totaal_gebruik', years)
compare_years_cumsum(daily, 'Net_gebruik', years)
compare_years_cumsum(daily, 'Terugleversaldo', years)

In [ ]:
# Views with daily granularity
compare_years(daily, 'Gas', years, unit = 'm3', lbl = lambda y, d: y + ' (' + str(round(sum(d.fillna(0)), 2)) + ' m3)')

power_lbl = lambda y, d: y + ' (' + str(round(sum(d.fillna(0)), 2)) + ' kWh)'
compare_years(daily, 'Totaal_gebruik', years, lbl = power_lbl)
compare_years(daily, 'Bruto_productie', years, lbl = power_lbl)
compare_years(daily, 'Teruglevering', years, lbl = power_lbl)
compare_years(daily, 'Net_gebruik', years, lbl = power_lbl)
compare_years(daily, 'Eigen_gebruik_productie_pct', years, unit = '%')
compare_years(daily, 'Terugleversaldo', years, lbl = power_lbl)

In [ ]:
# Weekly
weeks = lambda d: d.index.week
power_lbl = lambda y, d: y + ' (' + str(round(sum(d.fillna(0)), 2)) + ' kWh / dag)'

compare_years(weekly_mean, 'Gas', years, unit = 'm3 / dag', x_axis = weeks)
compare_years(weekly_mean, 'Totaal_gebruik', years, lbl = power_lbl, x_axis = weeks)
compare_years(weekly_mean, 'Bruto_productie', years, lbl = power_lbl, x_axis = weeks)
compare_years(weekly_mean, 'Teruglevering', years, lbl = power_lbl, x_axis = weeks)
compare_years(weekly_mean, 'Net_gebruik', years, lbl = power_lbl, x_axis = weeks)
compare_years(weekly_mean, 'Eigen_gebruik_productie_pct', years, unit = '%', x_axis = weeks)
compare_years(weekly_mean, 'Terugleversaldo', years, lbl = power_lbl, x_axis = weeks)

In [ ]:
# Monthly
months = lambda d: d.index.month
power_lbl = lambda y, d: y + ' (' + str(round(sum(d.fillna(0)), 2)) + ' kWh / dag)'

compare_years(monthly_mean, 'Gas', years, unit = 'm3 / dag', x_axis = months)
compare_years(monthly_mean, 'Totaal_gebruik', years, lbl = power_lbl, x_axis = months)
compare_years(monthly_mean, 'Bruto_productie', years, lbl = power_lbl, x_axis = months)
compare_years(monthly_mean, 'Teruglevering', years, lbl = power_lbl, x_axis = months)
compare_years(monthly_mean, 'Net_gebruik', years, lbl = power_lbl, x_axis = months)
compare_years(monthly_mean, 'Eigen_gebruik_productie_pct', years, unit = '%', x_axis = months)
compare_years(monthly_mean, 'Terugleversaldo', years, lbl = power_lbl, x_axis = months)

In [ ]:
gas = daily.filter(['Maand', 'Weekdag', 'Gas'])
sns.boxplot(data=gas, x='Maand', y='Gas');

In [ ]:
sns.boxplot(data=gas, x='Weekdag', y='Gas');

In [ ]:
kwh = daily.filter(['Maand', 'Weekdag', 'Totaal_gebruik'])
kwh['Totaal_gebruik'] = kwh['Totaal_gebruik'].map(lambda x: x / 1000, na_action=None)

sns.boxplot(data=kwh, x='Maand', y='Totaal_gebruik');

In [ ]:
sns.boxplot(data=kwh, x='Weekdag', y='Totaal_gebruik');

In [ ]:
kwh = daily.filter(['Maand', 'Weekdag', 'Bruto_productie'])
kwh['Bruto_productie'] = kwh['Bruto_productie'].map(lambda x: x / 1000, na_action=None)
sns.boxplot(data=kwh, x='Maand', y='Bruto_productie');